In [1]:
import pandas as pd
from script import DataManager

dm = DataManager()

2022-03-24 10:06:31 WARN  NativeCodeLoader:62 - Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
[Stage 0:>                                                         (0 + 6) / 12]

2022-03-24 10:06:41 WARN  TaskSetManager:66 - Stage 0 contains a task of very large size (111 KB). The maximum recommended task size is 100 KB.


[Stage 1:=======================================>               (148 + 7) / 206]

2022-03-24 10:06:43 WARN  TaskSetManager:66 - Stage 1 contains a task of very large size (111 KB). The maximum recommended task size is 100 KB.


#### We now have access to several DataFrames - after running the cell above:

- `dm.train_df`  
    all *train-*.csv* files concatenated into 1 **DataFrame**
- `dm.spark_train_df`  
    a Spark Dataframe of all training data
- `dm.writing_df`  
    the *writing.json* file parsed into a **DataFrame**
- `dm.directing_df`  
    the *directing.json* file parsed into a **DataFrame**
- `dm.joined_df`   
    result of `writing_df` left-joined with `directing_df` on column *movie*)
- `df.validation_df`  
    the *validation_hidden.csv* file as a **DataFrame**
- `dm.spark_validation_df`  
    a Spark Dataframe of all validation data
- `dm.test_df`  
    the *test_hidden.csv* file as a **DataFrame**
- `dm.spark_test_df`  
    a Spark Dataframe of all test data

## THE MOVIE DATABASE API


### This API is completely independent from IMDb, since it has its own database. With the code below we can find additional data on each movie. We split up the work in three parts.

##### Getting all movie IDs


> 1) First we need all the *IMDB_IDs* which we extract over all three datasets

##### Making a mapping from IMDB to TMDB

> 2) Using these *IMDB_IDs* we can get the *TMDB_ID* that corresponds to this movie using the API. If such a mapping already exists we don't send out the request. The complete mapping is saved in _imdb/data/api/imdb_to_tmdb_ids.json_.



#### Getting the actual data for each movie

> 3) Using the *TMDB_IDs* we can get extra data, which comes back as a JSON. I have selected the columns we already want from the beginning and loaded in a country information dataset. This dataset maps a country code to the region where it resides. This way we can see where the movie is made. If there is no mapping possible this is being set to `World`. The genres are collected and a DataFrame is created. We eventually write it away to the _parquet_ format.  

##### Steps 1, 2 and 3 are made easier implemented using the DataManager.

IMPORTANT: All scraped data can then be found in **dm.scraped_data_location**, which is saved as a lightweight *parquet* file.

In [2]:
# step 1 and 2 
dm.update_imdb_to_tmdb_id_mapper()

2022-03-24 10:06:57 WARN  TaskSetManager:66 - Stage 7 contains a task of very large size (111 KB). The maximum recommended task size is 100 KB.


100%|██████████| 10000/10000 [00:08<00:00, 1220.95it/s]


In [3]:
# step 3
dm.update_scraped_movie_data()

100%|██████████| 9954/9954 [00:00<00:00, 64688.72it/s]


In [4]:
print(dm.spark_train_df.count())
print(dm.train_df.shape)

print(dm.spark_validation_df.count())
print(dm.validation_df.shape)

print(dm.spark_test_df.count())
print(dm.test_df.shape)

2022-03-24 10:07:15 WARN  TaskSetManager:66 - Stage 10 contains a task of very large size (111 KB). The maximum recommended task size is 100 KB.
7959
(7959, 8)
955
(955, 7)
1086
(1086, 7)


### ML Pipeline using PySpark functionality 
---------------

##### With the additional data in place we can train our model(s).

> 1) We first prepare the datasets, joining what needs to be joines and changing column datatypes for example.  

> 2) We then create the Transformer and Estimator objects available from the __pyspark.ml__ library, and put this together in a Pipeline object.    

> 3) With the Pipeline we can fit our model.  

> 4) We can then make predictions and save them.

In [12]:
# dm.X_train.printSchema()
# dm.X_train.withColumn("runtimeMinutes",dm.X_train.runtimeMinutes.cast(FloatType()))

In [7]:
type(dm.X_train)

pyspark.sql.dataframe.DataFrame

In [8]:
print((dm.X_train.count(), len(dm.X_train.columns)))

2022-03-24 10:10:13 WARN  TaskSetManager:66 - Stage 49 contains a task of very large size (111 KB). The maximum recommended task size is 100 KB.
(7924, 34)


In [6]:
# this step creates dm.X_train, dm.X_validation and dm.X_test
dm.prepare_datasets()

rfc_pipeline = dm.setup_ml_pipeline()
trained_model = dm.fit_model(rfc_pipeline)

dm.make_predictions(
    model=trained_model, 
    unseen_data=dm.X_validation, 
    original_left=dm.validation_df.copy(), 
    filename_for_saving="validation_TESTRUN.csv"
)

2022-03-24 10:09:51 WARN  Utils:66 - Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.debug.maxToStringFields' in SparkEnv.conf.


2022-03-24 10:09:53 WARN  TaskSetManager:66 - Stage 18 contains a task of very large size (111 KB). The maximum recommended task size is 100 KB.


2022-03-24 10:09:55 WARN  TaskSetManager:66 - Stage 20 contains a task of very large size (111 KB). The maximum recommended task size is 100 KB.
2022-03-24 10:09:57 WARN  TaskSetManager:66 - Stage 25 contains a task of very large size (111 KB). The maximum recommended task size is 100 KB.
2022-03-24 10:09:57 WARN  TaskSetManager:66 - Stage 27 contains a task of very large size (111 KB). The maximum recommended task size is 100 KB.
2022-03-24 10:09:58 WARN  TaskSetManager:66 - Stage 29 contains a task of very large size (111 KB). The maximum recommended task size is 100 KB.
2022-03-24 10:09:58 WARN  TaskSetManager:66 - Stage 30 contains a task of very large size (111 KB). The maximum recommended task size is 100 KB.
2022-03-24 10:09:59 WARN  TaskSetManager:66 - Stage 31 contains a task of very large size (111 KB). The maximum recommended task size is 100 KB.


2022-03-24 10:10:01 WARN  TaskSetManager:66 - Stage 33 contains a task of very large size (111 KB). The maximum recommended task size is 100 KB.


2022-03-24 10:10:02 WARN  TaskSetManager:66 - Stage 35 contains a task of very large size (111 KB). The maximum recommended task size is 100 KB.
2022-03-24 10:10:02 WARN  TaskSetManager:66 - Stage 37 contains a task of very large size (111 KB). The maximum recommended task size is 100 KB.
2022-03-24 10:10:02 WARN  TaskSetManager:66 - Stage 39 contains a task of very large size (111 KB). The maximum recommended task size is 100 KB.
2022-03-24 10:10:02 WARN  TaskSetManager:66 - Stage 41 contains a task of very large size (111 KB). The maximum recommended task size is 100 KB.
2022-03-24 10:10:03 WARN  TaskSetManager:66 - Stage 43 contains a task of very large size (111 KB). The maximum recommended task size is 100 KB.
